# Segmentación - DIP

## Configuración del ambiente

In [ ]:
# Descarga de las imágenes con las que vamos a trabajar
!wget -q https://github.com/MoraRubio/dip-uam/raw/main/img/lenna.jpg https://github.com/MoraRubio/dip-uam/raw/main/img/pcb.png https://github.com/MoraRubio/dip-uam/raw/main/img/building.jpg https://github.com/MoraRubio/dip-uam/raw/main/img/raptor.png https://github.com/MoraRubio/dip-uam/raw/main/img/circuit.png

In [ ]:
# Importamos las librerías necesarias
import cv2
import matplotlib.pyplot as plt
import numpy as np
from skimage import data

%matplotlib inline

def plot_images(img, title=None, font_size=None, axis="off", color=cv2.COLOR_BGR2RGB):
    n_imgs = len(img)
    if  n_imgs > 1:
        _, axs = plt.subplots(1, n_imgs, **{'figsize':(3*n_imgs, 3)})
        axs = axs.ravel()
        for i in range(n_imgs):
            if title and (len(title) == n_imgs):
                axs[i].set_title(title[i], fontsize=font_size)
            axs[i].axis(axis)
            if len(img[i].shape) == 2:
              axs[i].imshow(img[i], cmap="gray")
            else:
              axs[i].imshow(cv2.cvtColor(img[i], color))
        plt.tight_layout()    
    else:
        plt.title(title, fontsize=font_size)
        plt.axis(axis)
        if len(img[0].shape) == 2:
          plt.imshow(img[0], cmap="gray")
        else:
          plt.imshow(cv2.cvtColor(img[0], color))

In [ ]:
lenna = cv2.resize(cv2.imread("../dip-uam/img/lenna.jpg", cv2.IMREAD_GRAYSCALE), (512, 512))
pcb = cv2.resize(cv2.imread("../dip-uam/img/pcb.png", cv2.IMREAD_GRAYSCALE), (512, 512))
building = cv2.imread("../dip-uam/img/building.jpg", cv2.IMREAD_GRAYSCALE)
brick = data.brick()
cell = data.cell()
_, binary_raptor = cv2.threshold(cv2.imread("../dip-uam/img/raptor.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
_, binary_circuit = cv2.threshold(cv2.imread("../dip-uam/img/circuit.png", cv2.IMREAD_GRAYSCALE), 127, 255, cv2.THRESH_BINARY)
plot_images([lenna, pcb, building, brick, cell, binary_raptor, binary_circuit])

## Detección de puntos aislados

In [ ]:
laplacian = np.array([[-1, -1, -1],
                      [-1,  8, -1],
                      [-1, -1, -1]], dtype=np.float32)

pcb_lap = cv2.filter2D(pcb[20:100,250:350], -1, laplacian)
plot_images([pcb[20:100,250:350], pcb_lap, pcb_lap > 50])

## Detección de líneas

In [ ]:
laplacian = np.array([[1, 1, 1],
                      [1,-8, 1],
                      [1, 1, 1]], dtype=np.float32)

circuit_lap = cv2.filter2D(binary_circuit, -1, laplacian)
plot_images([binary_circuit, circuit_lap])

### Líneas verticales

In [ ]:
def vertical_line_detector(size):
    kernel = np.full((size, size), fill_value=-1)
    center = size//2
    kernel[:,center] = 2
    return kernel

In [ ]:
vertical = vertical_line_detector(3)
circuit_lap1 = cv2.filter2D(binary_circuit, ddepth=cv2.CV_8U, kernel=vertical)
circuit_lap = cv2.filter2D(binary_circuit, ddepth=cv2.CV_64F, kernel=vertical)
plot_images([binary_circuit, circuit_lap, circuit_lap1])

### Líneas diagonales

In [ ]:
def diagonal_line_detector(size):
    kernel = np.full((size, size), fill_value=-1)
    np.fill_diagonal(kernel, 2)
    return kernel

In [ ]:
diagonal = diagonal_line_detector(7)
circuit_lap = cv2.filter2D(binary_circuit, -1, diagonal)
plot_images([binary_circuit, circuit_lap])

## Detección de bordes

### Básica
Utilizando operadores de gradiente o de primera derivada como los Operadores Sobel:

<img src="https://github.com/MoraRubio/dip-uam/blob/main/src/sobel.png?raw=true" alt="Kernels laplacianos" style="height: 200px; width:400px;"/>

In [ ]:
sobelx = cv2.Sobel(src=brick, ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=brick, ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=brick, ddepth=cv2.CV_8U, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([brick, sobelx, sobely, sobelxy], ["Original", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

In [ ]:
blurred_brick = cv2.blur(brick, (5, 5))

sobelx = cv2.Sobel(src=blurred_brick, ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=blurred_brick, ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=blurred_brick, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([blurred_brick, sobelx, sobely, sobelxy], ["Blurred", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

In [ ]:
sobelx = cv2.Sobel(src=building[:800,:800], ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=building[:800,:800], ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=building[:800,:800], ddepth=cv2.CV_8U, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([building[:800,:800], sobelx, sobely, sobelxy], ["Original", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

In [ ]:
sobelx = cv2.Sobel(src=building[400:600,400:500], ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=building[400:600,400:500], ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=building[400:600,400:500], ddepth=cv2.CV_8U, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([building[400:600,400:500], sobelx, sobely, sobelxy], ["Original", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

In [ ]:
blurred_building = cv2.blur(building, (5, 5))

sobelx = cv2.Sobel(src=blurred_building[:800,:800], ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=blurred_building[:800,:800], ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=blurred_building[:800,:800], ddepth=cv2.CV_8U, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([blurred_building[:800,:800], sobelx, sobely, sobelxy], ["Blurred", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

In [ ]:
sobelx = cv2.Sobel(src=cell, ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=cell, ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=cell, ddepth=cv2.CV_8U, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([cell, sobelx, sobely, sobelxy], ["Original", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

In [ ]:
sobelx = cv2.Sobel(src=lenna, ddepth=cv2.CV_8U, dx=1, dy=0, ksize=5) # Sobel Edge Detection on the X axis
sobely = cv2.Sobel(src=lenna, ddepth=cv2.CV_8U, dx=0, dy=1, ksize=5) # Sobel Edge Detection on the Y axis
sobelxy = cv2.Sobel(src=lenna, ddepth=cv2.CV_8U, dx=1, dy=1, ksize=5) # Combined X and Y Sobel Edge Detection

plot_images([lenna, sobelx, sobely, sobelxy], ["Original", "Sobel en la dirección x", "Sobel en la dirección y", "Sobel en la dirección x,y"])

### Métodos avanzados
[https://github.com/adl1995/edge-detectors](https://github.com/adl1995/edge-detectors)

#### El detector de bordes Marr-Hildreth
El algoritmo para este detector se puede resumir en:

1. Filtrar la imagen con un kernel pasabajas Gaussiano de n $\times$ n.
2. Aplicar el filtro laplaciano o derivada de segundo orden (Calcular el laplaciano de la imagen).
3. Encontrar los pasos por cero (*[zero-crossings](https://en.wikipedia.org/wiki/Zero_crossing)*) en el laplaciano.

In [ ]:
def zero_crossings(LoG, t):
    minLoG = cv2.morphologyEx(LoG, cv2.MORPH_ERODE, np.ones((3,3)))
    maxLoG = cv2.morphologyEx(LoG, cv2.MORPH_DILATE, np.ones((3,3)))
    return np.logical_or(np.logical_and(minLoG < -t,  LoG > t), np.logical_and(maxLoG > t, LoG < -t))

In [ ]:
gaussian_building = cv2.GaussianBlur(building, ksize=(3, 3), sigmaX=building.shape[0]*0.005)
laplacian_building = cv2.Laplacian(gaussian_building, ddepth=cv2.CV_32F, ksize=7)
marr_edges_building = zero_crossings(laplacian_building, laplacian_building.max()*0.04)
plot_images([building, gaussian_building, laplacian_building, marr_edges_building])
plot_images([building[:800,:800], gaussian_building[:800,:800], laplacian_building[:800,:800], marr_edges_building[:800,:800]])

In [ ]:
gaussian_lenna = cv2.GaussianBlur(lenna, ksize=(3, 3), sigmaX=lenna.shape[0]*0.005)
laplacian_lenna = cv2.Laplacian(gaussian_lenna, ddepth=cv2.CV_32F, ksize=3)
marr_edges_lenna = zero_crossings(laplacian_lenna, laplacian_lenna.max()*0.04)
plot_images([lenna, gaussian_lenna, laplacian_lenna, marr_edges_lenna])

In [ ]:
gaussian_cell = cv2.GaussianBlur(cell, ksize=(3, 3), sigmaX=cell.shape[0]*0.005)
laplacian_cell = cv2.Laplacian(gaussian_cell, ddepth=cv2.CV_32F, ksize=3)
marr_edges_cell = zero_crossings(laplacian_cell, laplacian_cell.max()*0.16)
plot_images([cell, gaussian_cell, laplacian_cell, marr_edges_cell])

#### Canny

In [ ]:
canny_building = cv2.Canny(building, 100, 300)
plot_images([building, canny_building])
plot_images([building[:800,:800], canny_building[:800,:800]])